#Aula 10 - Supercomputação 24.2
###Alexandre Santarossa

In [1]:
!nvidia-smi

Tue Sep 17 21:12:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Writing exemplo1.cu


In [ ]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1


In [ ]:
!./exemplo1


Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [ ]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [ ]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [ ]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [ ]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing stocks.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [ ]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00246407
Tempo de CÓPIA (em segundos)  0.105706


In [2]:
%%writefile exercicio1.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <iostream>
#include <fstream>
#include <chrono>

int main() {
    // Abra o arquivo de entrada
    std::ifstream infile("stocks-google.txt");
    if (!infile) {
        std::cerr << "Não foi possível abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    // Vetor temporário para armazenar os dados lidos
    std::vector<double> temp_data;
    double value;

    // Leia os dados do arquivo e armazene no vetor temporário
    while (infile >> value) {
        temp_data.push_back(value);
    }

    infile.close();

    // Crie um host_vector a partir dos dados lidos
    thrust::host_vector<double> host_data(temp_data.begin(), temp_data.end());

    // Medir o tempo de cópia dos dados para o device_vector
    auto start = std::chrono::steady_clock::now();

    // Crie um device_vector e copie os dados do host_vector
    thrust::device_vector<double> device_data = host_data;

    auto end = std::chrono::steady_clock::now();

    // Calcule o tempo de cópia
    std::chrono::duration<double> elapsed_seconds = end - start;
    std::cout << "Tempo de alocação e cópia para a GPU: " << elapsed_seconds.count() << " segundos" << std::endl;

    return 0;
}

Writing exercicio1.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 exercicio1.cu -o exercicio1

In [9]:
!./exercicio1

Tempo de alocação e cópia para a GPU: 0.302275 segundos


In [10]:
%%writefile exercicio2.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <fstream>

int main() {
    // Abra o arquivo de entrada
    std::ifstream infile("stocks-google.txt");
    if (!infile) {
        std::cerr << "Não foi possível abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    // Vetor temporário para armazenar os dados lidos
    std::vector<double> temp_data;
    double value;

    // Leia os dados do arquivo e armazene no vetor temporário
    while (infile >> value) {
        temp_data.push_back(value);
    }

    infile.close();

    // Crie um host_vector a partir dos dados lidos
    thrust::host_vector<double> host_data(temp_data.begin(), temp_data.end());

    // Crie um device_vector e copie os dados do host_vector
    thrust::device_vector<double> device_data = host_data;

    // Total de elementos
    size_t N = device_data.size();

    // Cálculo da média total
    double sum_total = thrust::reduce(device_data.begin(), device_data.end(), 0.0, thrust::plus<double>());
    double average_total = sum_total / N;

    std::cout << "Preço médio das ações do Google nos últimos 10 anos: " << average_total << std::endl;

    // Cálculo da média dos últimos 365 dias
    size_t last_year_days = 365;

    if (N >= last_year_days) {
        auto start_iter = device_data.end() - last_year_days;
        auto end_iter = device_data.end();

        double sum_last_year = thrust::reduce(start_iter, end_iter, 0.0, thrust::plus<double>());
        double average_last_year = sum_last_year / last_year_days;

        std::cout << "Preço médio das ações nos últimos 365 dias: " << average_last_year << std::endl;

        // Encontrar o maior e o menor preço no período total
        auto minmax_total = thrust::minmax_element(device_data.begin(), device_data.end());
        double min_price_total = *minmax_total.first;
        double max_price_total = *minmax_total.second;

        std::cout << "Menor preço no período total: " << min_price_total << std::endl;
        std::cout << "Maior preço no período total: " << max_price_total << std::endl;

        // Encontrar o maior e o menor preço nos últimos 365 dias
        auto minmax_last_year = thrust::minmax_element(start_iter, end_iter);
        double min_price_last_year = *minmax_last_year.first;
        double max_price_last_year = *minmax_last_year.second;

        std::cout << "Menor preço nos últimos 365 dias: " << min_price_last_year << std::endl;
        std::cout << "Maior preço nos últimos 365 dias: " << max_price_last_year << std::endl;
    } else {
        std::cout << "Dados insuficientes para os últimos 365 dias." << std::endl;
    }

    return 0;
}

Writing exercicio2.cu


In [11]:
!nvcc -arch=sm_75 -std=c++14 exercicio2.cu -o exercicio2

In [12]:
!./exercicio2

Preço médio das ações do Google nos últimos 10 anos: 1580.08
Preço médio das ações nos últimos 365 dias: 1615.81
Menor preço no período total: 1015.24
Maior preço no período total: 2200
Menor preço nos últimos 365 dias: 1060.5
Maior preço nos últimos 365 dias: 2200


In [20]:
%%writefile exercicio3.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <sstream>

int main() {
    // Abra o arquivo de entrada
    std::ifstream infile("stocks2.txt");
    if (!infile) {
        std::cerr << "Não foi possível abrir o arquivo stocks2.txt" << std::endl;
        return 1;
    }

    // Vetores temporários para armazenar os dados lidos
    std::vector<double> temp_data_AAPL;
    std::vector<double> temp_data_MSFT;
    std::string line;

    // Leia os dados do arquivo e armazene nos vetores temporários
    while (std::getline(infile, line)) {
        std::stringstream ss(line);
        std::string value_str;
        double value_AAPL, value_MSFT;

        // Leia o primeiro valor (AAPL)
        if (std::getline(ss, value_str, ',')) {
            value_AAPL = std::stod(value_str);
        } else {
            continue; // Linha inválida
        }

        // Leia o segundo valor (MSFT)
        if (std::getline(ss, value_str, ',')) {
            value_MSFT = std::stod(value_str);
        } else {
            continue; // Linha inválida
        }

        temp_data_AAPL.push_back(value_AAPL);
        temp_data_MSFT.push_back(value_MSFT);
    }

    infile.close();

    // Verificar se os tamanhos dos vetores são iguais
    if (temp_data_AAPL.size() != temp_data_MSFT.size()) {
        std::cerr << "Os vetores de dados têm tamanhos diferentes!" << std::endl;
        return 1;
    }

    size_t N = temp_data_AAPL.size();

    // Crie host_vectors a partir dos dados lidos
    thrust::host_vector<double> host_data_AAPL(temp_data_AAPL.begin(), temp_data_AAPL.end());
    thrust::host_vector<double> host_data_MSFT(temp_data_MSFT.begin(), temp_data_MSFT.end());

    // Crie device_vectors e copie os dados do host_vector
    thrust::device_vector<double> device_data_AAPL = host_data_AAPL;
    thrust::device_vector<double> device_data_MSFT = host_data_MSFT;

    // Criar um device_vector para armazenar as diferenças
    thrust::device_vector<double> device_diff(N);

    // Calcular a diferença ponto a ponto entre os dois vetores
    thrust::transform(device_data_AAPL.begin(), device_data_AAPL.end(),
                      device_data_MSFT.begin(), device_diff.begin(),
                      thrust::minus<double>());

    // Calcular a média das diferenças
    double sum_diff = thrust::reduce(device_diff.begin(), device_diff.end(), 0.0, thrust::plus<double>());
    double average_diff = sum_diff / N;

    std::cout << "A média das diferenças entre os preços das ações das empresas Apple e Microsoft é de " << average_diff << std::endl;

    if (average_diff > 0) {
        std::cout << "A Apple teve um preço maior." << std::endl;
    } else if (average_diff < 0) {
        std::cout << "A Microsoft teve um preço maior." << std::endl;
    } else {
        std::cout << "As duas empresas tiveram preços iguais." << std::endl;
    }

    return 0;
}

Overwriting exercicio3.cu


In [21]:
!nvcc -arch=sm_75 -std=c++14 exercicio3.cu -o exercicio3

In [22]:
!./exercicio3

A média das diferenças entre os preços das ações das empresas Apple e Microsoft é de -265.232
A Microsoft teve um preço maior.
